In [ ]:
!pip install ultralytics
!pip install boto3

In [ ]:
from ultralytics import YOLO
import boto3
import os
import torch
from glob import glob
from pathlib import Path

print(torch.cuda.is_available()) # comment this line if no gpu available

In [3]:
def download_data_from_s3(client, bucket_name, prefix, local_dir):
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)
    for obj in client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)['Contents']:
        if not os.path.exists(os.path.join(os.getcwd(), os.path.dirname(obj['Key']))):
            os.makedirs(os.path.dirname(obj['Key']))
        if obj['Key'][-1] != '/':
            client.download_file(bucket_name, obj['Key'], obj['Key'])

def get_keys(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        if len(lines) != 2:
            raise Exception("Unable to read keys. Invalid file format.")
        return (lines[0][:-1], lines[1])

id, secret = get_keys("aws_keys.txt")
download_data_from_s3(boto3.client('s3', aws_access_key_id=id, aws_secret_access_key=secret), "sagemaker-us-east-1-992382414571", "datasets", "datasets")

In [4]:
labels = glob(os.path.join(os.getcwd(), "datasets/bollards/labels/*/*.txt"), recursive=True)


root_dir = Path(labels[0]).parent.parent.parent
for label in labels:
    label = Path(label)
    subset = label.parent.name
    img_path = [(root_dir / "images" / subset / label.with_suffix(sfx).name)
                for sfx in [".png", ".jpg", ".PNG", ".JPG"]]
    img_path = [pth for pth in img_path if pth.exists()][0]
    boxes = []
    points = []
    classes = []
    save_pth = root_dir / "labels_kp" / subset / label.name
    save_pth.parent.mkdir(parents=True, exist_ok=True)
    with open(label) as f:
        lines = f.readlines()
        for line in lines:
            splits = line.rstrip().split(" ")
            cls_id = int(splits[0])
            box = splits[1:]
            if not box:
                with open(save_pth, "w") as f:
                    pass
                continue

            box = [float(pt) for pt in box]
            point = (box[0], box[1])
            points.append(point)
            boxes.append(box)
            classes.append(cls_id)

    with open(save_pth, "w") as f:
        for point, box, cls_id in zip(points, boxes, classes):
            f.writelines(f"{cls_id} {box[0]} {box[1]} {box[2]} {box[3]} {point[0]} {point[1]} 1 \n")


In [ ]:
model = YOLO('yolov8m-pose.pt')
model.to('cuda')

results = model.train(data="datasets/geoguessr.yaml", epochs=150,
                      imgsz=1920, rect=True, batch=4,
                      fliplr=0, 
                      mosaic=0, 
                      hsv_h=0, 
                      hsv_s=0, 
                      hsv_v=0, 
                      dropout=0.1)

In [ ]:
model_d = YOLO("ultralytics/cfg/models/v8/yolov8m.yaml").load(model.model)
model_d.ckpt = {"model": model_d.model}
model_d.save("best_detect.pt")
results = model_d.val(data="datasets/geoguessr.yaml")

In [ ]:
model = YOLO('yolov8m.pt')
model.to('cuda') # comment this line if no gpu available

results = model.train(data="datasets/geoguessr.yaml", 
                      batch=4,
                      epochs=150,  
                      imgsz=1920, 
                      rect=True, 
                      iou=0.7, 
                      visualize=True, 
                      augment=True, 
                      fliplr=0, 
                      mosaic=0, 
                      hsv_h=0, 
                      hsv_s=0, 
                      hsv_v=0, 
                      dropout=0.1)
results = model.val()

In [ ]:
model.export(imgsz=(1088, 1920))